# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 12 2022 1:15PM,244083,10,MTR-22-0244,"Senseonics, Incorporated",Released
1,Aug 12 2022 1:02PM,244079,10,Enova-10480,Emerginnova,Released
2,Aug 12 2022 1:02PM,244079,10,Enova-10481,Emerginnova,Released
3,Aug 12 2022 12:39PM,244078,21,624088,"NBTY Global, Inc.",Executing
4,Aug 12 2022 12:30PM,244077,10,8394514-SD,Eywa Pharma Inc.,Released
5,Aug 12 2022 12:04PM,244076,19,60017388,"GUSA Granules USA, Inc.",Released
6,Aug 12 2022 12:03PM,244069,20,ALUR966429621,Alumier Labs Inc.,Released
7,Aug 12 2022 12:03PM,244069,20,ALU0007464,Alumier Labs Inc.,Released
8,Aug 12 2022 12:03PM,244069,20,ALUR240979788,Alumier Labs Inc.,Released
9,Aug 12 2022 12:03PM,244069,20,ALUR628984819,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,244076,Released,1
27,244077,Released,1
28,244078,Executing,1
29,244079,Released,2
30,244083,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244076,NaN,1.0
244077,NaN,1.0
244078,1.0,NaN
244079,NaN,2.0
244083,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244001,0.0,1.0
244012,9.0,3.0
244014,0.0,1.0
244023,0.0,12.0
244024,5.0,19.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244001,0,1
244012,9,3
244014,0,1
244023,0,12
244024,5,19


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244001,0,1
1,244012,9,3
2,244014,0,1
3,244023,0,12
4,244024,5,19


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244001,,1
1,244012,9,3
2,244014,,1
3,244023,,12
4,244024,5,19


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 12 2022 1:15PM,244083,10,"Senseonics, Incorporated"
1,Aug 12 2022 1:02PM,244079,10,Emerginnova
3,Aug 12 2022 12:39PM,244078,21,"NBTY Global, Inc."
4,Aug 12 2022 12:30PM,244077,10,Eywa Pharma Inc.
5,Aug 12 2022 12:04PM,244076,19,"GUSA Granules USA, Inc."
6,Aug 12 2022 12:03PM,244069,20,Alumier Labs Inc.
14,Aug 12 2022 11:58AM,244073,20,"ACI Healthcare USA, Inc."
29,Aug 12 2022 11:57AM,244074,10,ISDIN Corporation
64,Aug 12 2022 11:44AM,244072,10,"Citieffe, Inc."
65,Aug 12 2022 11:21AM,244066,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 12 2022 1:15PM,244083,10,"Senseonics, Incorporated",,1
1,Aug 12 2022 1:02PM,244079,10,Emerginnova,,2
2,Aug 12 2022 12:39PM,244078,21,"NBTY Global, Inc.",1,
3,Aug 12 2022 12:30PM,244077,10,Eywa Pharma Inc.,,1
4,Aug 12 2022 12:04PM,244076,19,"GUSA Granules USA, Inc.",,1
5,Aug 12 2022 12:03PM,244069,20,Alumier Labs Inc.,,8
6,Aug 12 2022 11:58AM,244073,20,"ACI Healthcare USA, Inc.",,15
7,Aug 12 2022 11:57AM,244074,10,ISDIN Corporation,,35
8,Aug 12 2022 11:44AM,244072,10,"Citieffe, Inc.",,1
9,Aug 12 2022 11:21AM,244066,20,"ACI Healthcare USA, Inc.",1,43


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 12 2022 1:15PM,244083,10,"Senseonics, Incorporated",1,
1,Aug 12 2022 1:02PM,244079,10,Emerginnova,2,
2,Aug 12 2022 12:39PM,244078,21,"NBTY Global, Inc.",,1
3,Aug 12 2022 12:30PM,244077,10,Eywa Pharma Inc.,1,
4,Aug 12 2022 12:04PM,244076,19,"GUSA Granules USA, Inc.",1,
5,Aug 12 2022 12:03PM,244069,20,Alumier Labs Inc.,8,
6,Aug 12 2022 11:58AM,244073,20,"ACI Healthcare USA, Inc.",15,
7,Aug 12 2022 11:57AM,244074,10,ISDIN Corporation,35,
8,Aug 12 2022 11:44AM,244072,10,"Citieffe, Inc.",1,
9,Aug 12 2022 11:21AM,244066,20,"ACI Healthcare USA, Inc.",43,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 12 2022 1:15PM,244083,10,"Senseonics, Incorporated",1,
1,Aug 12 2022 1:02PM,244079,10,Emerginnova,2,
2,Aug 12 2022 12:39PM,244078,21,"NBTY Global, Inc.",,1
3,Aug 12 2022 12:30PM,244077,10,Eywa Pharma Inc.,1,
4,Aug 12 2022 12:04PM,244076,19,"GUSA Granules USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 12 2022 1:15PM,244083,10,"Senseonics, Incorporated",1.0,NaN
1,Aug 12 2022 1:02PM,244079,10,Emerginnova,2.0,NaN
2,Aug 12 2022 12:39PM,244078,21,"NBTY Global, Inc.",NaN,1.0
3,Aug 12 2022 12:30PM,244077,10,Eywa Pharma Inc.,1.0,NaN
4,Aug 12 2022 12:04PM,244076,19,"GUSA Granules USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 12 2022 1:15PM,244083,10,"Senseonics, Incorporated",1.0,0.0
1,Aug 12 2022 1:02PM,244079,10,Emerginnova,2.0,0.0
2,Aug 12 2022 12:39PM,244078,21,"NBTY Global, Inc.",0.0,1.0
3,Aug 12 2022 12:30PM,244077,10,Eywa Pharma Inc.,1.0,0.0
4,Aug 12 2022 12:04PM,244076,19,"GUSA Granules USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2440500,88.0,47.0
102,244014,1.0,0.0
15,244036,1.0,0.0
16,1708334,6.0,1.0
19,488133,1.0,1.0
20,976234,91.0,10.0
21,244078,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2440500,88.0,47.0
1,102,244014,1.0,0.0
2,15,244036,1.0,0.0
3,16,1708334,6.0,1.0
4,19,488133,1.0,1.0
5,20,976234,91.0,10.0
6,21,244078,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,88.0,47.0
1,102,1.0,0.0
2,15,1.0,0.0
3,16,6.0,1.0
4,19,1.0,1.0
5,20,91.0,10.0
6,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,102,Released,1.0
2,15,Released,1.0
3,16,Released,6.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,15,16,19,20,21
Status,,,,,,,
Executing,47.0,0.0,0.0,1.0,1.0,10.0,1.0
Released,88.0,1.0,1.0,6.0,1.0,91.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,15,16,19,20,21
0,Executing,47.0,0.0,0.0,1.0,1.0,10.0,1.0
1,Released,88.0,1.0,1.0,6.0,1.0,91.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,15,16,19,20,21
0,Executing,47.0,0.0,0.0,1.0,1.0,10.0,1.0
1,Released,88.0,1.0,1.0,6.0,1.0,91.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()